In [1]:
%env CUDA_VISIBLE_DEVICES=4,6
%env WORLD_SIZE=2

env: CUDA_VISIBLE_DEVICES=4,6
env: WORLD_SIZE=2


In [2]:
import deepspeed 
from magma.magma import Magma
from train import get_pretraining_datasets
from magma.utils import configure_param_groups, load_model
from magma.config import MultimodalConfig
from magma.datasets import collate_fn
from magma.config import MultimodalConfig
from functools import partial
from torch.optim import AdamW

Torch distributed Default Port: 29500


In [3]:
deepspeed.init_distributed()
model = Magma(
            'fb20-dgx2-configs/dev.yml'
        )
tokenizer, config, transforms = model.tokenizer, model.config, model.transforms
trainable_parameters = configure_param_groups(model, config)

# load data:
train_dataset, eval_dataset = get_pretraining_datasets(
    config, tokenizer, transforms
)

[2023-02-02 11:25:33,507] [INFO] [distributed.py:40:init_distributed] Not using the DeepSpeed or torch.distributed launchers, attempting to detect MPI environment...
[2023-02-02 11:25:34,430] [INFO] [distributed.py:87:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=130.83.42.209, master_port=29500
[2023-02-02 11:25:34,432] [INFO] [distributed.py:50:init_distributed] Initializing torch distributed with backend: nccl
Loading GPTJ language model...


loading dataset paths from /home/ml-mmeuer/adaptable_magma/magma/datasets/coco_train_val: 5000it [00:00, 361478.21it/s]
loading dataset paths from /home/ml-mmeuer/adaptable_magma/magma/datasets/coco_train_val: 5000it [00:00, 15532.00it/s]

Loaded train dataset with 5000 samples
Loaded eval dataset with 5000 samples


In [4]:
config.load

'/home/ml-mmeuer/adaptable_magma/model_checkpoints/v1_1'

In [5]:

config = MultimodalConfig.from_yml(
            'fb20-dgx2-configs/dev.yml'
        )
opt = AdamW(
        trainable_parameters,
        config.lr,
        betas=(0.9, 0.95),
        weight_decay=config.weight_decay,
        eps=1.0e-4
    )

model_engine, opt, train_loader, lr_scheduler = deepspeed.initialize(
        # args=config,
        model=model,
        optimizer=opt,
        model_parameters=trainable_parameters,
        training_data=train_dataset,
        collate_fn=partial(collate_fn, seq_len=model.seq_len),
        config_params=config.deepspeed_config_params,
    )
    
previous_global_step = load_model(
            model_engine,
            config.load,
            load_optimizer_states=config.load_optimizer,
            load_lr_scheduler_states=config.load_optimizer,
        )

[2023-02-02 11:27:07,452] [INFO] [logging.py:69:log_dist] [Rank 0] DeepSpeed info: version=0.5.10, git-hash=unknown, git-branch=unknown
[2023-02-02 11:27:17,167] [INFO] [logging.py:69:log_dist] [Rank 0] initializing deepspeed groups
[2023-02-02 11:27:17,174] [INFO] [logging.py:69:log_dist] [Rank 0] initializing deepspeed model parallel group with size 1
[2023-02-02 11:27:17,183] [INFO] [logging.py:69:log_dist] [Rank 0] initializing deepspeed expert parallel group with size 1
[2023-02-02 11:27:17,184] [INFO] [logging.py:69:log_dist] [Rank 0] creating expert data parallel process group with ranks: [0]
[2023-02-02 11:27:17,185] [INFO] [logging.py:69:log_dist] [Rank 0] creating expert parallel process group with ranks: [0]


/home/ml-mmeuer/anaconda3/envs/nlp/lib/python3.9/site-packages/torch/distributed/distributed_c10d.py:429: UserWarning: torch.distributed.distributed_c10d._get_global_rank is deprecated please use torch.distributed.distributed_c10d.get_global_rank instead
  warnings.warn(


[2023-02-02 11:27:17,604] [INFO] [engine.py:275:__init__] DeepSpeed Flops Profiler Enabled: False
[2023-02-02 11:27:17,605] [INFO] [engine.py:1091:_configure_optimizer] Removing param_group that has no 'params' in the client Optimizer
[2023-02-02 11:27:17,612] [INFO] [engine.py:1097:_configure_optimizer] Using client Optimizer as basic optimizer
[2023-02-02 11:27:17,633] [INFO] [engine.py:1113:_configure_optimizer] DeepSpeed Basic Optimizer = AdamW
[2023-02-02 11:27:17,635] [INFO] [utils.py:44:is_zero_supported_optimizer] Checking ZeRO support for optimizer=AdamW type=<class 'torch.optim.adamw.AdamW'>
[2023-02-02 11:27:17,635] [INFO] [logging.py:69:log_dist] [Rank 0] Creating fp16 ZeRO stage 2 optimizer
[2023-02-02 11:27:17,636] [INFO] [stage_1_and_2.py:121:__init__] Reduce bucket size 500000000
[2023-02-02 11:27:17,636] [INFO] [stage_1_and_2.py:122:__init__] Allgather bucket size 500000000
[2023-02-02 11:27:17,637] [INFO] [stage_1_and_2.py:123:__init__] CPU Offload: True
[2023-02-02 1

Using /home/ml-mmeuer/.cache/torch_extensions/py39_cu117 as PyTorch extensions root...
Emitting ninja build file /home/ml-mmeuer/.cache/torch_extensions/py39_cu117/utils/build.ninja...
Building extension module utils...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module utils...


ninja: no work to do.
Time to load utils op: 0.7421677112579346 seconds
Rank: 0 partition count [1] and sizes[(247619864, False)] 
[2023-02-02 11:27:21,790] [INFO] [utils.py:822:see_memory_usage] Before initializing optimizer states
[2023-02-02 11:27:21,793] [INFO] [utils.py:823:see_memory_usage] MA 12.13 GB         Max_MA 12.13 GB         CA 12.6 GB         Max_CA 13 GB 
[2023-02-02 11:27:21,794] [INFO] [utils.py:831:see_memory_usage] CPU Virtual Memory:  used = 67.57 GB, percent = 6.7%
[2023-02-02 11:27:23,232] [INFO] [utils.py:822:see_memory_usage] After initializing optimizer states
[2023-02-02 11:27:23,234] [INFO] [utils.py:823:see_memory_usage] MA 12.13 GB         Max_MA 12.13 GB         CA 12.6 GB         Max_CA 13 GB 
[2023-02-02 11:27:23,236] [INFO] [utils.py:831:see_memory_usage] CPU Virtual Memory:  used = 70.37 GB, percent = 7.0%
[2023-02-02 11:27:23,236] [INFO] [stage_1_and_2.py:493:__init__] optimizer state initialized
[2023-02-02 11:27:23,404] [INFO] [utils.py:822:see_me

Using /home/ml-mmeuer/.cache/torch_extensions/py39_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...


[2023-02-02 11:27:37,191] [INFO] [engine.py:2621:_get_all_zero_checkpoints] successfully read 2 ZeRO state_dicts for rank 0


ZeRORuntimeException: The checkpoint being loaded used a DP world size of 2 but the current world size is 1. Automatic adjustment of ZeRO's optimizer state partitioning with a new world size is not currently supported.

In [ ]:
import torch
from einops import rearrange
import json
import os
import clip
from PIL import Image
import sys
import inspect
from activations.torch import Rational

from activations.utils.get_weights import get_parameters
from clip.model import Bottleneck
from clip.clip import _transform
from magma.transforms import clip_preprocess
from magma.transforms import get_transforms

from pathlib import Path
# from .find_init
# _weights import find_weights
# from .warnings import RationalImportError
import numpy as np
from termcolor import colored
known_functions = {
    "relu": lambda x: 0 if x < 0 else x,
    "leaky_relu": lambda x: x/100 if x < 0 else x,
    "lrelu": lambda x: x/100 if x < 0 else x,
    "normal": lambda x: 1/np.sqrt(2*np.pi) * np.exp(-.5*x**2),
}
from magma.image_prefix import get_image_encoder

Torch distributed Default Port: 29500


In [ ]:
enc = get_image_encoder(
            "clip_resnet_large",
            convert_to_rational=True
        ).to("cuda")
transform_magma = get_transforms(1, 'clip_resnet_large', input_resolution=224)
transform_clip = _transform(224)


In [ ]:
def freeze_rational_clip(enc):
    for child in enc.children():
        if isinstance(child, torch.nn.Sequential):
            for seq_child in child.children():
                if isinstance(seq_child, Bottleneck):
                    for bot_params in seq_child.children():
                        if isinstance(bot_params, (Rational, torch.nn.BatchNorm2d)):
                            
                            for param in bot_params.parameters():
                                param.requires_grad = True

                        else:
                            for param in bot_params.parameters():
                                param.requires_grad = False
                else:
                    for param in seq_child.parameters():
                        param.requires_grad = False
        else:
            for param in child.parameters():
                param.requires_grad = False
    return enc

In [ ]:
enc = freeze_rational_clip(enc)

In [ ]:
img = Image.open("./magma/datasets/coco_train_val/images/0/000000000139.jpg").resize((224, 224))

In [ ]:
x_magma = transform_magma(img).to("cuda")
x_clip = transform_clip(img)[None, ...].to("cuda")

In [ ]:
enc(x_magma).isnan().any()
enc(x_clip).isnan().any()

tensor(False, device='cuda:0')

In [ ]:
prefix = []
bottleneck = []
for i , child in enumerate(list(ModifiedResNet.children())): 
    if i < 10:
        prefix.append(child)
    if i ==10:
        bottleneck.append(list(list(child.children())[0].children()))

NameError: name 'ModifiedResNet' is not defined

In [ ]:
prefix = torch.nn.Sequential(*prefix)
bottleneck = torch.nn.Sequential(*bottleneck[0])

In [ ]:
w_numerator, w_denominator = get_parameters("A", (5,4),
                                            "relu")

numerator = torch.nn.Parameter(torch.tensor(w_numerator, dtype=torch.float16),
                                True).to('cuda')

denominator = torch.nn.Parameter(torch.tensor(w_denominator, dtype=torch.float16),
                                requires_grad=True).to('cuda')


In [ ]:
normed_x = prefix(x)

In [ ]:
def Rational_PYTORCH_A_F(x, weight_numerator, weight_denominator):
    # P(X) / Q(X) = a_0 + a_1 * X + ... + a_n * X^n /
    #               1 + | b_1 * X | + | b_2 * X^2| + ... + | b_m * X ^m|
    x_32 = x.
    len_num, len_deno = len(weight_numerator), len(weight_denominator)
    pre = torch.tensor([1.]).to(
        device=x.device, dtype=torch.float32)  # .half()
    post = torch.zeros(len(weight_numerator) -
                       len(weight_denominator) - 1).to(device=x.device, dtype=torch.float32)

    z = x_32.view(-1)
    singles = z.view(z.shape[-1], 1)
    pre_vander = singles.repeat(1, max(len_num, len_deno))
    pows = torch.arange(0, max(len_num, len_deno),
                        device=x.device, dtype=x.dtype)
    torch.nan_to_num(pre_vander, nan=0, posinf=torch.finfo(x.dtype).max, neginf=torch.finfo(x.dtype).min)
    vander = torch.pow(pre_vander, pows)
    print("vander: ", vander.isinf().any() or vander.isnan().any())
    numerator = torch.mul(vander, weight_numerator).sum(-1)

    expanded_dw = torch.cat([pre, weight_denominator, post])

    denominator = torch.mul(vander, expanded_dw).abs().sum(-1)

    flat_out = torch.div(numerator, denominator)

    out = flat_out.view(x_32.shape).to(x.dtype)
    print("out: ", out.isinf().any() or out.isnan().any())
    return out


In [ ]:
Rational_PYTORCH_A_F(normed_x,numerator, denominator).isinf().any()

vander:  tensor(False, device='cuda:0')
out:  tensor(False, device='cuda:0')


tensor(False, device='cuda:0')

In [ ]:
input  = (prefix(x))
for i, (name,child) in enumerate(bottleneck.named_children()):
     print("-"*10)
     print(name)
     out = child(input)
     print(out.min(), out.max())
     if i ==1:
          normed_x = out
     print(child.weight.dtype)
     input = out
    

----------
0
tensor(-0.7773, device='cuda:0', dtype=torch.float16) tensor(0.5835, device='cuda:0', dtype=torch.float16)
torch.float16
----------
1
tensor(-8.4375, device='cuda:0', dtype=torch.float16, grad_fn=<MinBackward1>) tensor(4.3711, device='cuda:0', dtype=torch.float16, grad_fn=<MaxBackward1>)
torch.float32
----------
2
pows:  tensor(False, device='cuda:0')
tensor(-1.8037, device='cuda:0', dtype=torch.float16, grad_fn=<MinBackward1>) tensor(4.3008, device='cuda:0', dtype=torch.float16, grad_fn=<MaxBackward1>)


AttributeError: 'Rational' object has no attribute 'weight'

In [ ]:
bottleneck(prefix(x))

RuntimeError: Given groups=1, weight of size [256, 64, 1, 1], expected input[1, 256, 56, 56] to have 64 channels, but got 256 channels instead

In [ ]:
after_pre = torch.randn(prefix(x).shape, dtype=torch.float16).to('cuda')

In [ ]:
list(list(list(mod_resnet.children())[10].children())[0].children())

[Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Rational,
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Rational,
 Identity(),
 Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
 BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Rational,
 Sequential(
   (-1): AvgPool2d(kernel_size=1, stride=1, padding=0)
   (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
   (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 )]

In [ ]:
ModifiedResNet

ModifiedResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): Rational
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [ ]:
ModifiedResNet = list(enc.named_modules())[0][1]


In [ ]:
input = x
for name, module in ModifiedResNet.named_modules(): 
    print("-"*10)
    print(module)
    out = module(input)
    print(out)
    input = out
    

----------
ModifiedResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): Rational
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), st

RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1, 49, 2048] to have 3 channels, but got 1 channels instead

In [ ]:
for i,(name, module) in enumerate(list(enc.named_modules())): 

    initial_layers = []
    firs_bottle_neck = []
    if i < 11: 
        print("-"*10)
        print(name, module)
        
        initial_layers.append(module)
    elif i > 10 and i < 49:
        firs_bottle_neck.append(module)
    

----------
 ModifiedResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): Rational
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), s

In [ ]:
firs_bottle_neck

[]